In [2]:
from utils import simulate_tournament, simulate_n_tournaments, compare_settings


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

In [3]:
n_tournaments = 800
n_teams = 8
n_rounds = 4
strategies = {'e':[1]}

compare_settings(n_tournaments, n_teams, n_rounds, strategies, probabilistic=True, deterministic=True)


Probabilistic resolution


,Level,Strategy,Strategic_avg_WR,Control_avg_WR,Delta
a,1.0,[],0.694063,0.67625,0.017813
b,0.875,[],0.662188,0.637813,0.024375
c,0.75,[],0.614375,0.621563,-0.007188
d,0.625,[],0.585313,0.579688,0.005625
f,0.375,[],0.4525,0.4475,0.005
e,0.5,[1],0.404687,0.493437,-0.08875
g,0.25,[],0.362187,0.355,0.007188
h,0.125,[],0.224688,0.18875,0.035938


---------------------------------------------------
Deterministic resolution


,Level,Strategy,Strategic_avg_WR,Control_avg_WR,Delta
a,1.0,[],1.0,1.0,0.0
b,0.875,[],0.75,0.75,0.0
c,0.75,[],0.65375,0.665625,-0.011875
d,0.625,[],0.558125,0.5475,0.010625
e,0.5,[1],0.448437,0.439375,0.009062
f,0.375,[],0.329375,0.3475,-0.018125
g,0.25,[],0.22375,0.25,-0.02625
h,0.125,[],0.036562,0.0,0.036562
